### Exercise 1-2. 
The steel model of this chapter can be further modified to reflect various changes in production requirements. For each part below, explain the modifications to Figures 1-6a and 1-6b that
would be required to achieve the desired changes. (Make each change separately, rather than accumulating the changes from one part to the next.)

(a) How would you change the constraints so that total hours used by all products must equal the total hours available for each stage? Solve the linear program with this change, and verify that you get the same results. Explain why, in this case, there is no difference in the solution.

(b) How would you add to the model to restrict the total weight of all products to be less than a new parameter, max_weight? Solve the linear program for a weight limit of 6500 tons, and explain how this extra restriction changes the results.

(c) The incentive system for mill managers may tend to encourage them to produce as many tons as possible. How would you change the objective function to maximize total tons? For the data of our example, does this make a difference to the optimal solution?
    
(d) Suppose that instead of the lower bounds represented by commit[p] in our model, we want to require that each product represent a certain share of the total tons produced. In the algebraic notation of Figure 1-1, this new constraint might be represented as 
    
\begin{equation*}
X_j >= s_j \thinspace \sum_{k \thinspace \epsilon \thinspace P} X_k \thinspace , for \thinspace each \thinspace  j \thinspace \epsilon \thinspace P
\end{equation*}
        
where sj is the minimum share associated with project j. How would you change the AMPL model to use this constraint in place of the lower bounds commit[p]? If the minimum shares are 0.4 for bands and plate, and 0.1 for coils, what is the solution ? Verify that if you change the minimum shares to 0.5 for bands and plate, and 0.1 for coils, the linear program gives an optimal solution that produces nothing, at zero profit. Explain why this makes sense.
      
(e) Suppose there is an additional finishing stage for plates only, with a capacity of 20 hours and a rate of 150 tons per hour. Explain how you could modify the data, without changing the model, to incorporate this new stage.


In [1]:
import pandas as pd
from amplpy import AMPL, Environment

In [2]:
ampl = AMPL(Environment('/opt/ampl.linux64'))

In [3]:
ampl.reset()

In [4]:
ampl.read('./ex--1-2a.mod')

In [5]:
ampl.readData('./ex--1-2a.dat')

In [6]:
variables = ('bands','coils','plate')
time_constraints = ('reheat','roll')

In [7]:
print(ampl.getConstraint('Time'))
#print(ampl.getConstraint('Time').get('reheat'))
#print(ampl.getConstraint('Time').get('roll'))
print()
print('Variables and their bounds')
for i in ampl.getVariables():
    [print(i[1].get(j)) for j in variables]
print()
print('Constraints and their limits')
for i in ampl.getConstraints():
    if i[0] == 'Time': [print(i[1].get(j)) for j in time_constraints]
    if i[0] == 'max_weight': print(i[1].get())
print()
print('Objective Function')
print(ampl.getObjective('Total_Profit').get())

subject to Time{s in Stage} : sum{p in Products} 1/tonsPerHour[p,s]*X[p]
   <= maxHours[s];

Variables and their bounds
var X['bands'] >=1000, <=6000;
var X['coils'] >=500, <=4000;
var X['plate'] >=750, <=3500;

Constraints and their limits
subject to Time['reheat']:
	0.005*X['bands'] + 0.005*X['coils'] + 0.005*X['plate'] <= 35;
subject to Time['roll']:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;

Objective Function
maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];


In [8]:
ampl.eval('expand Total_Profit;')
ampl.eval('expand Time;')

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];

subject to Time['reheat']:
	0.005*X['bands'] + 0.005*X['coils'] + 0.005*X['plate'] <= 35;

subject to Time['roll']:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;



In [9]:
for variable in variables:
    ampl.eval('expand X["{}"];'.format(variable))

Coefficients of X['bands']:
	Time['reheat']   0.005
	Time['roll']     0.005
	Total_Profit    25

Coefficients of X['coils']:
	Time['reheat']   0.005
	Time['roll']     0.00714286
	Total_Profit    30

Coefficients of X['plate']:
	Time['reheat']   0.005
	Time['roll']     0.00625
	Total_Profit    29



In [10]:
ampl.eval('display _nvars, _ncons;')

_nvars = 3
_ncons = 2



In [11]:
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))

minos
cplex


In [12]:
ampl.solve()

CPLEX 12.8.0.0: optimal solution; objective 190071.4286
2 dual simplex iterations (0 in phase I)


In [15]:
print(ampl.getObjective('Total_Profit').get().value())

190071.42857142858


#### Shadow Price or Dual Price or Marginal Price

In [16]:
ampl.eval('display Total_Profit;')
for i in ampl.getConstraints():
    print(i[1].getValues())
#print(ampl.getConstraint('Time').getValues())

Total_Profit = 190071

   index0    |  Time.dual  
  'reheat'   | 1799.9999999999973
   'roll'    | 3200.0000000000018



### Decision Variable Solution and Reduced Cost

In [17]:
X_result = list()
for p in variables:
    X_result.append([
        p,
        ampl.getVariable('X').get(p).lb(), 
        ampl.getVariable('X').get(p).value(),
        ampl.getVariable('X').get(p).ub(),
        ampl.getVariable('X').get(p).rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
bands,1000.0,3357.142857,6000.0,3.552714e-15
coils,500.0,500.000000,4000.0,-1.857143e+00
plate,750.0,3142.857143,3500.0,0.000000e+00


In [18]:
print(ampl.getData('X'))
values = ampl.getVariable('X').getValues()
print(values)
df = values.toPandas()
print(values.toList())
print(values.toDict())

   index0    |      X      
  'bands'    | 3357.1428571428542
  'coils'    |     500     
  'plate'    | 3142.8571428571458

   index0    |    X.val    
  'bands'    | 3357.1428571428542
  'coils'    |     500     
  'plate'    | 3142.8571428571458

[('bands', 3357.142857142854), ('coils', 500.0), ('plate', 3142.857142857146)]
{'bands': 3357.142857142854, 'coils': 500.0, 'plate': 3142.857142857146}


In [19]:
df['X.reheatRate'] = 1/200
df['X.rollRate'] = [1/i for i in (200,140,160)]
df['X.reheat']=df['X.val']*df['X.reheatRate']
df['X.roll']=df['X.val']*df['X.rollRate']
df

,X.val,X.reheatRate,X.rollRate,X.reheat,X.roll
bands,3357.142857,0.005,0.005000,16.785714,16.785714
coils,500.000000,0.005,0.007143,2.500000,3.571429
plate,3142.857143,0.005,0.006250,15.714286,19.642857


In [20]:
print('Reheat:{} Roll:{}'.format(df['X.reheat'].sum(),df['X.roll'].sum()))
print('Total Production = {} tons'.format(df['X.val'].sum()))

Reheat:35.0 Roll:40.0
Total Production = 7000.0 tons


## Summary

(a) Both constraints were binding in the original case where they were inequality (less than or equal to) constraints. We can see the constraints are already at their limit. Now making these constraints an equality constraint has no change to the solution because, we just forced the constraints to be at their limit always.